Vamos analisar cafe em grao sem torra com codigo CNM 0901.11.10 de acordo com https://portalunico.siscomex.gov.br/classif/#/nomenclatura/0901/expandida?h=cafe&palavraInteira=false

Para ETL vamos primeiro buscar os dados somente para isso na balança de importação e exportação


Como é .gov pedia autenticação, nesse caso baixei o .zip para ETL. 

In [21]:
import pandas as pd

# Set the chunk size
chunk_size = 1000000  # Adjust this based on your memory capacity

# Read the first chunk to inspect the columns
chunk = pd.read_csv('IMP_COMPLETA.csv', chunksize=chunk_size)

# Get the columns of the first chunk
first_chunk = next(chunk)  # Get the first chunk
print(first_chunk.columns)  # Print the column names

Index(['CO_ANO;"CO_MES";"CO_NCM";"CO_UNID";"CO_PAIS";"SG_UF_NCM";"CO_VIA";"CO_URF";"QT_ESTAT";"KG_LIQUIDO";"VL_FOB";"VL_FRETE";"VL_SEGURO"'], dtype='object')


In [22]:
import pandas as pd

# Set the chunk size
chunk_size = 1000000  # Adjust this based on your memory capacity

# Read the first chunk with the correct delimiter (semicolon)
chunk_iter = pd.read_csv('IMP_COMPLETA.csv', chunksize=chunk_size, delimiter=';')

# Get the columns of the first chunk
first_chunk = next(chunk_iter)  # Get the first chunk
print(first_chunk.columns)  # Print the column names

# Now proceed with the search after checking the columns
for chunk in pd.read_csv('IMP_COMPLETA.csv', chunksize=chunk_size, delimiter=';'):
    if 9011110 in chunk['CO_NCM'].values:
        resultado = chunk[chunk['CO_NCM'] == 9011110]
        print(resultado)
        break


Index(['CO_ANO', 'CO_MES', 'CO_NCM', 'CO_UNID', 'CO_PAIS', 'SG_UF_NCM',
       'CO_VIA', 'CO_URF', 'QT_ESTAT', 'KG_LIQUIDO', 'VL_FOB', 'VL_FRETE',
       'VL_SEGURO'],
      dtype='object')
        CO_ANO  CO_MES   CO_NCM  CO_UNID  CO_PAIS SG_UF_NCM  CO_VIA  CO_URF  \
261177    1997       3  9011110       21      586        PR       7  910600   
735062    1997       3  9011110       21      249        SP       1  817800   
862044    1997       2  9011110       21      586        PR       7  910600   

        QT_ESTAT  KG_LIQUIDO  VL_FOB  VL_FRETE  VL_SEGURO  
261177     30000       30000   90000       300          0  
735062       171      170859  284392     22250       1777  
862044        50       50400   89040       460          0  


In [26]:
import pandas as pd

# Caminho para o arquivo CSV
csv_file_path = 'IMP_COMPLETA.csv'

# CO_NCM específico que você deseja filtrar
co_ncm_especifico = "9011110"

# Nome do arquivo de saída
output_file = f"filtered_{csv_file_path.split('/')[-1]}"

# Tamanho do chunk
chunk_size = 1000000  # Ajuste conforme necessário

# Criar o arquivo de saída
with open(output_file, mode='w', newline='', encoding='utf-8') as output_csv:
    writer = None  # Variável para controlar a escrita do cabeçalho
    # Ler o CSV em chunks
    for chunk in pd.read_csv(csv_file_path, chunksize=chunk_size, delimiter=';', encoding='latin-1'):
        # Filtrar o chunk pelo valor de CO_NCM
        filtered_chunk = chunk[chunk['CO_NCM'].astype(str) == co_ncm_especifico]
        
        # Se o arquivo de saída estiver vazio, escrever o cabeçalho
        if writer is None:
            filtered_chunk.to_csv(output_csv, header=True, index=False, sep=';')
            writer = True  # Garantir que a próxima vez não escreva o cabeçalho
        else:
            # Caso contrário, escrever apenas as linhas filtradas
            filtered_chunk.to_csv(output_csv, header=False, index=False, sep=';')

print(f"Processamento concluído. Registros filtrados salvos em: {output_file}")


Processamento concluído. Registros filtrados salvos em: filtered_IMP_COMPLETA.csv


In [27]:
import pandas as pd

# Caminho para o arquivo CSV
csv_file_path = 'EXP_COMPLETA.csv'

# CO_NCM específico que você deseja filtrar
co_ncm_especifico = "9011110"

# Nome do arquivo de saída
output_file = f"filtered_{csv_file_path.split('/')[-1]}"

# Tamanho do chunk
chunk_size = 1000000  # Ajuste conforme necessário

# Criar o arquivo de saída
with open(output_file, mode='w', newline='', encoding='utf-8') as output_csv:
    writer = None  # Variável para controlar a escrita do cabeçalho
    # Ler o CSV em chunks
    for chunk in pd.read_csv(csv_file_path, chunksize=chunk_size, delimiter=';', encoding='latin-1'):
        # Filtrar o chunk pelo valor de CO_NCM
        filtered_chunk = chunk[chunk['CO_NCM'].astype(str) == co_ncm_especifico]
        
        # Se o arquivo de saída estiver vazio, escrever o cabeçalho
        if writer is None:
            filtered_chunk.to_csv(output_csv, header=True, index=False, sep=';')
            writer = True  # Garantir que a próxima vez não escreva o cabeçalho
        else:
            # Caso contrário, escrever apenas as linhas filtradas
            filtered_chunk.to_csv(output_csv, header=False, index=False, sep=';')

print(f"Processamento concluído. Registros filtrados salvos em: {output_file}")

Processamento concluído. Registros filtrados salvos em: filtered_EXP_COMPLETA.csv


In [31]:
import pandas as pd

# Caminho para os arquivos CSV
imp_file_path = 'filtered_IMP_COMPLETA.csv'
exp_file_path = 'filtered_EXP_COMPLETA.csv'

# Ler os dois arquivos CSV
imp_df = pd.read_csv(imp_file_path, delimiter=';', encoding='latin-1')
exp_df = pd.read_csv(exp_file_path, delimiter=';', encoding='latin-1')

# Adicionar a coluna 'Label' para identificar se é Import ou Export
imp_df['Label'] = 'Import'
exp_df['Label'] = 'Export'

# Concatenar os dois dataframes
merged_df = pd.concat([imp_df, exp_df], ignore_index=True)

# Exibir as primeiras linhas do dataframe concatenado
print(merged_df.head())

# Salvar o dataframe concatenado em um novo arquivo CSV, se necessário
merged_df.to_csv('merged_data_with_labels.csv', index=False, sep=';', encoding='utf-8')


   CO_ANO  CO_MES   CO_NCM  CO_UNID  CO_PAIS SG_UF_NCM  CO_VIA  CO_URF  \
0    1997       3  9011110       21      586        PR       7  910600   
1    1997       3  9011110       21      249        SP       1  817800   
2    1997       2  9011110       21      586        PR       7  910600   
3    2000       3  9011110       21      586        PR       7  910600   
4    2000       9  9011110       21       97        AC       7  230151   

   QT_ESTAT  KG_LIQUIDO  VL_FOB  VL_FRETE  VL_SEGURO   Label  
0     30000       30000   90000     300.0        0.0  Import  
1       171      170859  284392   22250.0     1777.0  Import  
2        50       50400   89040     460.0        0.0  Import  
3        75       75000  107813     375.0        0.0  Import  
4       248       17360    2480     100.0        0.0  Import  


PARA AMANHA, ENTENDER MELHOR ESSES DADOS, PENSAR EM COMO AVALIAR< ACHOQUE KG LIQUIDO E DANDO MERGE POR MES/ANO. 

CONFERIR AS TAABELAS QUE TINHA VISTO E FAZER ANALISE DE DADOS/ML
